In [ ]:
from functools import partial, reduce
from os import rename, makedirs
import os
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime
from typing import Literal

In [ ]:
# carpeta = f"../output/AGROPE1/definitivos"

# csv_files = [x for x in os.listdir(carpeta) if ".csv" in x]

# for f in csv_files: 
#     new = f"{carpeta}/{f.replace('_old','')}"
#     old = f"{carpeta}/{f}"
#     os.rename(old, new)


In [ ]:
# def gsheet_download_csv(id: str, target: str, url_template="https://docs.google.com/spreadsheets/d/{id}/export?format=csv"):
#     url = url_template.format(id=id)
#     return urllib.request.urlretrieve(url, target)

# gsheet_download_csv('1UyfAnRCes0OAOUR0aARfkfegTsUl4Puw', 'nomenclador.csv')
nomenclador = pd.read_csv('./geonomenclador.csv')
pais = {k:v for k,v in nomenclador[['geocodigo', 'desc_fundar']].iloc}
# continentes = {k:v for k,v in nomenclador[['codigo_fundar', 'continente_fundar']].iloc}
# nivel_agregacion = {k:v for k,v in nomenclador[['codigo_fundar', 'nivel_agregacion']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [ ]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, lista_cambios = lista_cambios):
    codes_set = set(map.keys())
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join([str(x) for x in list(diff_set)])
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if (new_col and desc_col):
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, creando la columna {new_col} y borrando la columna {desc_col}"
        elif desc_col:
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, cambiando los valores de la columna {desc_col}"
        else: 
            string = f"Usando los valores de la columna {code_col}, se verificó que todos los códigos se encuentran en el nomenclador vigente al {today}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

def cast_col(df:pd.DataFrame, col:str, cast_to:Literal['int','float','object'], lista_cambios=lista_cambios):
    old_type = str(df[col].dtype)
    df[col] = df[col].astype(cast_to)
    lista_cambios.append(f"El tipo de dato de la columna {col} fue cambiado {old_type} ->> {cast_to}")
    return df

def cast_colx(col, cast_to):
    return lambda df: cast_col(df=df, col=col, cast_to=cast_to)

PRECIO_g1.csv

In [11]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

sector


,sector,valor
0,Alimentos y bebidas no alcoholicas,26.93381
1,Bebidas alcoholicas y tabaco,3.49540
2,Prendas de vestir y calzado,9.90447
3,Vivienda agua electricidad gas y otros combust...,9.43438
4,Equipamiento y mantenimiento del hogar,6.36303


In [12]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys, value_name="_valor"),
    sort_vals_ascx(primary_keys),
    renombrar({'_valor': 'valor'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,sector,indicador,valor
0,Alimentos y bebidas no alcoholicas,valor,26.93381
1,Bebidas alcoholicas y tabaco,valor,3.49540
2,Bienes y servicios varios,valor,3.54986
3,Comunicaciones,valor,2.82317
4,Educacion,valor,2.29781


In [13]:
diccionario_cambios

{'PRECIO_g01': ['Long estricto: [sector, valor] -> [sector, _valor, indicador]',
  'Ordené las filas por las columnas: sector',
  'Se renombraron las columnas: _valor:valor']}

In [14]:
exportar(_df)

('../output/PRECIO1/definitivos/PRECIO_g01.csv',
 '../output/PRECIO1/definitivos/PRECIO_g01_old.csv')

PRECIO_g2.csv

In [15]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

region sector


,region,sector,valor
0,Nacional,Alimentos y bebidas no alcoholicas,26.9
1,GBA,Alimentos y bebidas no alcoholicas,23.4
2,Pampeana,Alimentos y bebidas no alcoholicas,28.6
3,NEA,Alimentos y bebidas no alcoholicas,35.3
4,NOA,Alimentos y bebidas no alcoholicas,34.7


In [16]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys, value_name="_valor"),
    sort_vals_ascx(primary_keys),
    renombrar({'_valor': 'valor'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,region,sector,indicador,valor
0,Cuyo,Alimentos y bebidas no alcoholicas,valor,28.4
1,Cuyo,Bebidas alcoholicas y tabaco,valor,3.6
2,Cuyo,Bienes y servicios varios,valor,3.6
3,Cuyo,Comunicaciones,valor,2.5
4,Cuyo,Educacion,valor,2.2


In [17]:
diccionario_cambios

{'PRECIO_g01': ['Long estricto: [sector, valor] -> [sector, _valor, indicador]',
  'Ordené las filas por las columnas: sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g02': ['Long estricto: [region, sector, valor] -> [region, sector, _valor, indicador]',
  'Ordené las filas por las columnas: region, sector',
  'Se renombraron las columnas: _valor:valor']}

In [18]:
exportar(_df)

('../output/PRECIO1/definitivos/PRECIO_g02.csv',
 '../output/PRECIO1/definitivos/PRECIO_g02_old.csv')

PRECIO_g3.csv

In [19]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,inflacion_todos,inflacion_positivos
0,1935,14.088050,14.088050
1,1936,0.000000,NaN
2,1937,2.315325,2.315325
3,1938,-0.646552,NaN
4,1939,5.965293,5.965293


In [21]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys ),
    sort_vals_ascx(primary_keys)
    
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1935,inflacion_todos,14.088050
1,1935,inflacion_positivos,14.088050
2,1936,inflacion_positivos,NaN
3,1936,inflacion_todos,0.000000
4,1937,inflacion_todos,2.315325


In [22]:
diccionario_cambios

{'PRECIO_g01': ['Long estricto: [sector, valor] -> [sector, _valor, indicador]',
  'Ordené las filas por las columnas: sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g02': ['Long estricto: [region, sector, valor] -> [region, sector, _valor, indicador]',
  'Ordené las filas por las columnas: region, sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g03': ['Long estricto: [anio, inflacion_todos, inflacion_positivos] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [23]:
exportar(_df)

('../output/PRECIO1/definitivos/PRECIO_g03.csv',
 '../output/PRECIO1/definitivos/PRECIO_g03_old.csv')

PRECIO_g4.csv

In [24]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 4

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3


,pais,iso3,inflacion_prom_07_22
0,Afganistán,AFG,5.464714
1,Albania,ALB,2.498375
2,Argelia,DZA,5.089000
3,Angola,AGO,16.764688
4,Argentina,ARG,35.981509


In [25]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais'] ),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,pais,indicador,valor
0,AFG,Afganistán,inflacion_prom_07_22,5.464714
1,AGO,Angola,inflacion_prom_07_22,16.764688
2,ALB,Albania,inflacion_prom_07_22,2.498375
3,ARE,Emiratos Árabes Unidos,inflacion_prom_07_22,2.390188
4,ARG,Argentina,inflacion_prom_07_22,35.981509


In [26]:
diccionario_cambios

{'PRECIO_g01': ['Long estricto: [sector, valor] -> [sector, _valor, indicador]',
  'Ordené las filas por las columnas: sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g02': ['Long estricto: [region, sector, valor] -> [region, sector, _valor, indicador]',
  'Ordené las filas por las columnas: region, sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g03': ['Long estricto: [anio, inflacion_todos, inflacion_positivos] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'PRECIO_g04': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-05-20, cambiando los valores de la columna pais',
  'Long estricto: [pais, iso3, inflacion_prom_07_22] -> [iso3, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3']}

In [27]:
exportar(_df)

('../output/PRECIO1/definitivos/PRECIO_g04.csv',
 '../output/PRECIO1/definitivos/PRECIO_g04_old.csv')

PRECIO_g5.csv

In [28]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 5

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,pais,iso3,anio,inflacion
0,Argentina,ARG,2000,-0.733699
1,Argentina,ARG,2001,-1.543428
2,Argentina,ARG,2002,40.952869
3,Argentina,ARG,2003,3.656320
4,Argentina,ARG,2004,6.100982


In [29]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais,indicador,valor
0,AFG,2000,Afganistán,inflacion,NaN
1,AFG,2001,Afganistán,inflacion,NaN
2,AFG,2002,Afganistán,inflacion,NaN
3,AFG,2003,Afganistán,inflacion,6.5
4,AFG,2004,Afganistán,inflacion,13.3


In [30]:
diccionario_cambios

{'PRECIO_g01': ['Long estricto: [sector, valor] -> [sector, _valor, indicador]',
  'Ordené las filas por las columnas: sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g02': ['Long estricto: [region, sector, valor] -> [region, sector, _valor, indicador]',
  'Ordené las filas por las columnas: region, sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g03': ['Long estricto: [anio, inflacion_todos, inflacion_positivos] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'PRECIO_g04': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-05-20, cambiando los valores de la columna pais',
  'Long estricto: [pais, iso3, inflacion_prom_07_22] -> [iso3, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3'],
 'PRECIO_g05': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-05

In [31]:
exportar(_df)

('../output/PRECIO1/definitivos/PRECIO_g05.csv',
 '../output/PRECIO1/definitivos/PRECIO_g05_old.csv')

PRECIO_g6.csv

In [32]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 6

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

grupo_de_paises anio


,grupo_de_paises,anio,mediana_paises_inflacion
0,América Latina y Caribe,1980,21.206
1,América Latina y Caribe,1981,17.300
2,América Latina y Caribe,1982,20.687
3,América Latina y Caribe,1983,16.783
4,América Latina y Caribe,1984,25.100


In [43]:
list(filter(lambda x: "América Latina" in x[1], pais.items()))

[('F5205', 'América Latina y El Caribe'),
 ('LAC', 'América Latina y el Caribe (excluidos Países de Altos Ingresos)'),
 ('LCN', 'América Latina y el Caribe'),
 ('TLA', 'América Latina y el Caribe (IDA y BIRF)')]

In [33]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,grupo_de_paises,anio,indicador,valor
0,América Latina y Caribe,1980,mediana_paises_inflacion,21.206
1,América Latina y Caribe,1981,mediana_paises_inflacion,17.300
2,América Latina y Caribe,1982,mediana_paises_inflacion,20.687
3,América Latina y Caribe,1983,mediana_paises_inflacion,16.783
4,América Latina y Caribe,1984,mediana_paises_inflacion,25.100


In [34]:
diccionario_cambios

{'PRECIO_g01': ['Long estricto: [sector, valor] -> [sector, _valor, indicador]',
  'Ordené las filas por las columnas: sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g02': ['Long estricto: [region, sector, valor] -> [region, sector, _valor, indicador]',
  'Ordené las filas por las columnas: region, sector',
  'Se renombraron las columnas: _valor:valor'],
 'PRECIO_g03': ['Long estricto: [anio, inflacion_todos, inflacion_positivos] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'PRECIO_g04': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-05-20, cambiando los valores de la columna pais',
  'Long estricto: [pais, iso3, inflacion_prom_07_22] -> [iso3, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3'],
 'PRECIO_g05': ['Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-05

In [35]:
exportar(_df)

('../output/PRECIO1/definitivos/PRECIO_g06.csv',
 '../output/PRECIO1/definitivos/PRECIO_g06_old.csv')

PRECIO_g7.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 7

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g8.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 8

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g9.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 9

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g10.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 10

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g11.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
# m49_cod = paises[['m49_code_unsd', 'iso3_desc_fundar']].dropna(how="any").sort_values(by="m49_code_unsd").reset_index(drop=True)
# m49_cod['m49_code_unsd'] = m49_cod['m49_code_unsd'].astype('int')
# pais_m49_cod = {k:v for k,v in m49_cod.iloc}
# codes_set_m49_cod = set(pais_m49_cod.keys())


In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g12.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 12

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g13.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 13

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g14.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 14

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g15.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 15

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g16.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 16

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g17.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 17

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g18.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 18

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g12.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 19

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g20.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 20

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g21.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 21

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g22.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 22

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g23.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 23

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g24.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 24

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

PRECIO_g25.csv

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 25

archivo = f"{SUBTOP}_g{str(grafico_n).zfill(2)}"
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

In [ ]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

In [ ]:
diccionario_cambios

In [ ]:
exportar(_df)

In [ ]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

In [ ]:
import json
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

with open(f'../output/PRECIO1/definitivos/paso_a_definitivos_{today}.json', 'w') as fp:
    json.dump(diccionario_cambios, fp, indent=4, ensure_ascii=False)
